## Black-Jack Game

### Global Values

In [1]:
## GLOBAL VARIABLE OF VALUES

import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 
          'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

playing = True

### Card Class

In [2]:
class Card():
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
#         self.value = values[rank]
    
    def __str__(self):
        return self.rank +" of "+ self.suit        

### Deck Class

In [3]:
class Deck():
    
    def __init__(self):
        self.deck = []
        for suit in suits:
            for rank in ranks:
                card = Card(suit, rank)
                self.deck.append(card)
                
    def __str__(self):
        card_deck = ''
        for card in self.deck:
            card_deck += "\n" + card.__str__()
        return "THE DECK HAS : "+card_deck
    
    def shuffle_deck(self):
        random.shuffle(self.deck)
    
    def deal(self):
        single_card = self.deck.pop()
        return single_card

In [15]:
test_deck = Deck()
test_deck.shuffle_deck()
print(test_deck)

### Hand Class

In [4]:
class Hand():
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0  # KEEP TRACK OF ACES
    
    def add_card(self,card):
        # card passed in from Deck.deal -> Single Card Object and it has its (suit, rank)
        self.cards.append(card)
        self.value += values[card.rank]
        
        # Track Aces
        if card.rank == "Ace":
            self.aces += 1
    
    def adjust_for_ace(self):
        
        # IF TOTAL VALUE > 21 and I STILL HAVE AN ACE
        # CHANGE MY ACE TO 1 INSTEAD OF 11
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [5]:
test_deck = Deck()
test_deck.shuffle_deck()

# PLAYER
test_player = Hand()
# DEAL or GRAB 1 card from the DECK
pulled_card = test_deck.deal()
print(pulled_card)
test_player.add_card(pulled_card)
print(test_player.value)

Six of Hearts
6


In [6]:
test_player.add_card(test_deck.deal())
print(test_player.value)

16


### Chips Class

In [7]:
class Chips():
    
    def __init__(self, total=100):
        self.total = total
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

### Implement function to play the game

#### Function for taking bets

In [8]:
def take_bet(chips):
    while True:
        try:
            chips.bet = int(input('How many times would you like to bet? : '))
        except:
            print("Sorry, please provide a number")
        else:
            if chips.bet > chips.total:
                print("You do not have enough chips, You have {}".format(chips.total))
            else:
                break

#### Function for taking hits

In [9]:
def hit(deck, hand):
    single_card = deck.deal()
    hand.add_card(single_card)
    hand.adjust_for_ace()

#### Function prompting the Player to Hit or Stand

In [10]:
def hit_or_stand(deck, hand):
    global playing
    
    while True:
        x = input("Hit or Stand? Enter h or s : ")
        
        if x[0].lower() == 'h':
            hit(deck, hand)
        elif x[0].lower() == 's':
            print("Player Stands, Dealer's Turn")
            playing = False
        else:
            print("Sorry! I did not understand that, Please enter h or s only!")
            continue
        
        break

#### Function to display the cards

In [14]:
def show_some(player, dealer):
    print("DEALERS HAND:")
    print('One card hidden!')
    print(dealer.cards[1])
    print('\nPLAYERS HAND:')
    for card in player.cards:
        print(card)

def show_all(player, dealer):
    print("DEALERS HAND:")
    for card in dealer.cards:
        print(card)
    print('\nPLAYERS HAND:')
    for card in player.cards:
        print(card)

#### Function to handle end of game scenarios

In [15]:
def player_busts(player, dealer, chips):
    print("BUST PLAYER")
    chips.lose_bet()

def player_wins(player, dealer, chips):
    print("PLAYER WINS!!")
    chips.win_bet()

def dealer_busts(player, dealer, chips):
    print("PLAYER WINS! DEALER BUSTED!!")
    chips.win_bet()

def dealer_wins(player, dealer, chips):
    print("DEALER WINS!!")
    chips.lose_bet()

def push(player, dealer):
    print("Dealer and Player Tie!! PUSH")


### Game Logic

In [20]:
while True:
    # Print an Opening Statement
    print("WELCOME TO BLACKJACK!!\n\n")
    
    # Create and Shuffle the deck and deal 2 cards to each Player
    deck = Deck()
    deck.shuffle_deck()
    
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Set up Player's chips
    player_chips = Chips()
    
    # Prompt the Player for their bet
    take_bet(player_chips)
    
    # Show cards (Keep One dealer card hidden)
    show_some(player_hand, dealer_hand)
    
    while playing:
        
        # Prompt for player to Hit or Stand
        hit_or_stand(deck, player_hand)
        
        # Show Cards (Keep one dealer card hidden)
        show_some(player_hand, dealer_hand)
        
        # If Player's hand exceeds 21, run player_busts() and beak out of the loop
        if player_hand.value > 21:
            player_busts(player_hand, dealer_hand, player_chips)
            
            break
    
    
    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    if player_hand.value <= 21:
        
        while dealer_hand.value < player_hand.value:
            hit(deck, dealer_hand)
    
        # Show all cards
        show_all(player_hand, dealer_hand)
        
        # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand, dealer_hand, player_chips)
        elif dealer_hand.value < player_hand.value:
            player_wins(player_hand, dealer_hand, player_chips)
        else:
            push(player_hand, dealer_hand)
            
    # Inform Player of their Chips total
    print("\nPlayer's total chips are at : {}".format(player_chips.total))
    
    # Ask to Play Again
    new_game = input("Would you like to play another hand? y/n : ")
    
    if new_game[0].lower() == 'y':
        playing = True
        continue
    else:
        print("Thank you for playing!")
        break

WELCOME TO BLACKJACK!!


How many times would you like to bet? : 1
DEALERS HAND:
One card hidden!
Two of Diamonds

PLAYERS HAND:
Ace of Hearts
Jack of Spades
Hit or Stand? Enter h or s : h
DEALERS HAND:
One card hidden!
Two of Diamonds

PLAYERS HAND:
Ace of Hearts
Jack of Spades
Six of Clubs
Hit or Stand? Enter h or s : s
Player Stands, Dealer's Turn
DEALERS HAND:
One card hidden!
Two of Diamonds

PLAYERS HAND:
Ace of Hearts
Jack of Spades
Six of Clubs
DEALERS HAND:
Seven of Spades
Two of Diamonds
Ten of Hearts

PLAYERS HAND:
Ace of Hearts
Jack of Spades
Six of Clubs
DEALER WINS!!

Player's total chips are at : 99
Would you like to play another hand? y/n : n
Thank you for playing!
